In [2]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder\
        .master("spark://192.168.2.97:7077") \
        .appName("Q3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.4.0-3.3")\
        .config("spark.cores.max", 2)\
        .getOrCreate()

spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/10 11:32:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/10 11:32:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/10 11:32:19 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [12]:
redComm1 = spark_session.read.json("../RC_2008-03")
redComm2 = spark_session.read.json("../RC_2008-04")

In [13]:
redComm1.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [14]:
subs = redComm1.select('subreddit')
subs.take(5)

[Row(subreddit='reddit.com'),
 Row(subreddit='reddit.com'),
 Row(subreddit='reddit.com'),
 Row(subreddit='reddit.com'),
 Row(subreddit='reddit.com')]

In [21]:
subCountM1 = subs\
    .groupBy("subreddit")\
    .count()\
    .orderBy("count", ascending=False)

In [19]:
subCountM2 = redComm2\
    .select('subreddit')\
    .groupBy('subreddit')\
    .count()\
    .orderBy('count', ascending=False)

In [24]:
subCountM1.show()

+-------------+------+
|    subreddit| count|
+-------------+------+
|   reddit.com|207092|
|     politics| 83375|
|  programming| 34642|
|         pics| 25352|
|      science| 22874|
|     business| 14555|
|    worldnews| 12725|
|        funny| 10223|
|entertainment|  9403|
|       gaming|  9313|
|         nsfw|  4036|
|          WTF|  3276|
|      offbeat|  2709|
|   technology|  2645|
|    Economics|  2313|
|        obama|  1863|
|      gadgets|  1714|
|  environment|  1323|
|    AskReddit|  1133|
|       comics|  1029|
+-------------+------+
only showing top 20 rows



In [29]:
subCountM2.join(subCountM1, Seq("subreddit","count"), "left").show(false)
subCountM2.join(subCountM1, Seq("subreddit","count"), "right").show(false)

NameError: name 'Seq' is not defined

In [30]:
spark_session.stop()